In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.4.1


In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images.shape

(60000, 28, 28)

In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=ca59d800e3330caa654887296be1a945cc1143d8e83ffc8a6cec462625bfb249
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=8ac138c70f3787eed117ab339ddfd8bb62012dea2492b0a4e075986f5c0e6bd7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.9118333458900452

Best val_accuracy So Far: 0.9118333458900452
Total elapsed time: 00h 01m 53s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        23072     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                884784    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 909,146
Trainable params: 909,146
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1502 - accuracy: 0.9445 - val_loss: 0.2776 - val_accuracy: 0.9058
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1115 - accuracy: 0.9591 - val_loss: 0.2572 - val_accuracy: 0.9162
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0801 - accuracy: 0.9706 - val_loss: 0.2847 - val_accuracy: 0.9107
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0592 - accuracy: 0.9778 - val_loss: 0.3312 - val_accuracy: 0.9158
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0447 - accuracy: 0.9841 - val_loss: 0.3682 - val_accuracy: 0.9118
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0343 - accuracy: 0.9881 - val_loss: 0.4011 - val_accuracy: 0.9163
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0268 - accuracy: 0.9901 - val_loss: 0.4798 - val_accuracy

In [19]:
model.predict(test_images)[0]

array([6.6918246e-15, 6.1326604e-16, 1.1063224e-12, 2.8508641e-16,
       2.8972931e-17, 6.8717566e-11, 3.8805054e-21, 3.7330377e-08,
       8.8553055e-19, 1.0000000e+00], dtype=float32)